In [1]:
from langgraph.graph import END, StateGraph, add_messages
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
from typing import TypedDict, Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.1, max_tokens=1000)
memory = MemorySaver()

class ChatbotState(TypedDict):
    messages : Annotated[list, add_messages]

search_tool = TavilySearchResults()
tools = [search_tool]

llm_with_tools = llm.bind_tools(tools=tools)
# print(llm_with_tools.invoke("what is current weather in bangalore?"))

def chat_bot_node(state: ChatbotState) -> ChatbotState:
    """
    A simple chatbot node that appends a response to the messages in the state.
    
    Args:
        state (ChatbotState): The current state of the chatbot.
        
    Returns:
        ChatbotState: The updated state with the chatbot's response appended.
    """
    
    return {"messages": llm_with_tools.invoke(state['messages'])}

def tools_router(state: ChatbotState):
    last_message = state['messages'][-1]
    if (hasattr(last_message, 'tool_calls') and len(last_message.tool_calls) > 0):
        return "tool_node"
    else:
        return END
    
tool_node = ToolNode(tools = tools)

flow = StateGraph(ChatbotState)
flow.add_node("chat_bot", chat_bot_node)
flow.add_node("tool_node", tool_node)
flow.set_entry_point("chat_bot")
flow.add_conditional_edges("chat_bot", tools_router)
flow.add_edge("tool_node", "chat_bot")

app = flow.compile(checkpointer=memory,interrupt_before=["tool_node"])

In [ ]:
config = {"configurable": {
    "thread_id": 1
}}


events = app.stream(
    {
        "messages": [HumanMessage(content="What is the weather in Bangalore?")]
    },
    config=config, 
    stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()  # Print the last message content



================================ Human Message =================================

What is the weather in Bangalore?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_tJQ6YWXlmcUNHCb5wXRMRlEn)
 Call ID: call_tJQ6YWXlmcUNHCb5wXRMRlEn
  Args:
    query: Bangalore weather


In [6]:

events = app.stream(None,config=config, stream_mode="values")

for event in events:
    event["messages"][-1].pretty_print()  # Print the last message content

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_tJQ6YWXlmcUNHCb5wXRMRlEn)
 Call ID: call_tJQ6YWXlmcUNHCb5wXRMRlEn
  Args:
    query: Bangalore weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Bengaluru Weather Forecast 4 Jun 2025 - Times of India", "url": "https://timesofindia.indiatimes.com/weather/bengaluru-weather-forecast-today/560001", "content": "Today's Weather in Bengaluru: In Bengaluru today, the weather is expected to be Partly Cloudy with a maximum temperature of 28°C and a minimum of 22°C.", "score": 0.95791626}, {"title": "Weather in Bangalore in April 2025 (State of Karnātaka)", "url": "https://world-weather.info/forecast/india/bangalor/april-2025/", "content": "*   [7 +90°+73°](https://world-weather.info/forecast/india/bangalor/07-april/)\n*   [8 +90°+73°](https://world-weather.info/forecast/india/bangalor/0